<a href="https://colab.research.google.com/github/GavinHacker/ml_practice/blob/master/recbyfm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
!pip install pymysql

    100% |████████████████████████████████| 51kB 2.0MB/s 


In [0]:
import pandas as pd
import pymysql
import pymysql.cursors
from functools import reduce
import numpy as np
import pandas as pd
import uuid
import datetime
#from pyfm import pylibfm
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics.pairwise import pairwise_distances
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

In [44]:
df = pd.read_csv('/content/drive/My Drive/colabnotebook/comment_origin_data_1.csv',sep='\t',encoding='utf-8')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
#df_data.to_csv('/content/drive/My Drive/colabnotebook/comment_origin_data.csv', sep='\t', encoding='utf-8')

In [51]:
df.shape

(389689, 29)

In [0]:
df = df.drop_duplicates()

In [0]:
df = df.drop(['Unnamed: 0'], axis=1)

In [54]:
df['ID'].shape

(289261,)

In [55]:
df.shape

(289261, 28)

In [0]:
df = df.drop_duplicates(['ID'])

In [57]:
df.head().T

,0,1,2,3,4
ID,10000,100231,205834,221461,225496
TIME,2015-10-09 20:02:19,2009-01-31 22:36:03,2016-11-25 17:19:18,2016-01-03 09:56:48,2016-03-20 22:54:09
MOVIEID,1.05339e+07,1.29483e+06,2.29392e+07,2.58641e+07,2.63036e+07
RATING,4,4,2,4,4
CONTENT,虽然依然充满了想象力，但突然觉得这部很一般呢。,开头很好听～～a fool in love,特别没意思了。,虽然也是浩瀚宇宙，但跟诺兰想要传达的完全不一样。全片证明了一个论题：学好数理化，走遍宇宙都不...,不寒而栗
CREATOR,妖怪dodo,妖怪dodo,妖怪dodo,妖怪dodo,妖怪dodo
ADD_TIME_x,2018-01-05 15:23:18,2018-01-19 16:16:23,2018-01-25 13:44:26,2018-01-25 15:55:03,2018-01-25 16:18:51
NEWDATA,0,0,0,0,0
USERID,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f
id,10533913,1294829,22939161,25864085,26303622


In [58]:
df.head()

,ID,TIME,MOVIEID,RATING,CONTENT,CREATOR,ADD_TIME_x,NEWDATA,USERID,id,...,enable,description,img,userid,rmax,rmin,ravg,rcount,rsum,rmedian
0,10000,2015-10-09 20:02:19,10533913.0,4,虽然依然充满了想象力，但突然觉得这部很一般呢。,妖怪dodo,2018-01-05 15:23:18,0,cf2349f9c01f9a5cd4050aebd30ab74f,10533913,...,1.0,NaN,NaN,cf2349f9c01f9a5cd4050aebd30ab74f,5.0,2.0,3.85714,7.0,27.0,4.0
1,100231,2009-01-31 22:36:03,1294829.0,4,开头很好听～～a fool in love,妖怪dodo,2018-01-19 16:16:23,0,cf2349f9c01f9a5cd4050aebd30ab74f,1294829,...,1.0,NaN,NaN,cf2349f9c01f9a5cd4050aebd30ab74f,5.0,2.0,3.85714,7.0,27.0,4.0
2,205834,2016-11-25 17:19:18,22939161.0,2,特别没意思了。,妖怪dodo,2018-01-25 13:44:26,0,cf2349f9c01f9a5cd4050aebd30ab74f,22939161,...,1.0,NaN,NaN,cf2349f9c01f9a5cd4050aebd30ab74f,5.0,2.0,3.85714,7.0,27.0,4.0
3,221461,2016-01-03 09:56:48,25864085.0,4,虽然也是浩瀚宇宙，但跟诺兰想要传达的完全不一样。全片证明了一个论题：学好数理化，走遍宇宙都不...,妖怪dodo,2018-01-25 15:55:03,0,cf2349f9c01f9a5cd4050aebd30ab74f,25864085,...,1.0,NaN,NaN,cf2349f9c01f9a5cd4050aebd30ab74f,5.0,2.0,3.85714,7.0,27.0,4.0
4,225496,2016-03-20 22:54:09,26303622.0,4,不寒而栗,妖怪dodo,2018-01-25 16:18:51,0,cf2349f9c01f9a5cd4050aebd30ab74f,26303622,...,1.0,NaN,NaN,cf2349f9c01f9a5cd4050aebd30ab74f,5.0,2.0,3.85714,7.0,27.0,4.0


In [0]:
df = df.drop(['CONTENT'], axis=1)

In [60]:
df.head().T

,0,1,2,3,4
ID,10000,100231,205834,221461,225496
TIME,2015-10-09 20:02:19,2009-01-31 22:36:03,2016-11-25 17:19:18,2016-01-03 09:56:48,2016-03-20 22:54:09
MOVIEID,1.05339e+07,1.29483e+06,2.29392e+07,2.58641e+07,2.63036e+07
RATING,4,4,2,4,4
CREATOR,妖怪dodo,妖怪dodo,妖怪dodo,妖怪dodo,妖怪dodo
ADD_TIME_x,2018-01-05 15:23:18,2018-01-19 16:16:23,2018-01-25 13:44:26,2018-01-25 15:55:03,2018-01-25 16:18:51
NEWDATA,0,0,0,0,0
USERID,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f
id,10533913,1294829,22939161,25864085,26303622
name,头脑特工队,拜见岳父大人,星际迷航3：超越星辰,火星救援,大空头


In [0]:
df = df.drop(['ADD_TIME_x', 'ADD_TIME_y'], axis=1)

In [73]:
df.head().T

,0,1,2,3,4
ID,10000,100231,205834,221461,225496
TIME,2015-10-09 20:02:19,2009-01-31 22:36:03,2016-11-25 17:19:18,2016-01-03 09:56:48,2016-03-20 22:54:09
MOVIEID,1.05339e+07,1.29483e+06,2.29392e+07,2.58641e+07,2.63036e+07
RATING,4,4,2,4,4
CREATOR,妖怪dodo,妖怪dodo,妖怪dodo,妖怪dodo,妖怪dodo
NEWDATA,0,0,0,0,0
USERID,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f
id,10533913,1294829,22939161,25864085,26303622
name,头脑特工队,拜见岳父大人,星际迷航3：超越星辰,火星救援,大空头
type,剧情|奇幻|冒险|喜剧,剧情|喜剧|爱情,奇幻|战争|剧情|科幻|冒险|动作,剧情|灾难|冒险|喜剧|科幻,剧情|犯罪|传记|喜剧|历史


In [63]:
df.shape

(289228, 25)

In [64]:
df.iloc[10000]

ID                                       226336
TIME                        2016-12-12 18:59:32
MOVIEID                             2.64166e+07
RATING                                        5
CREATOR                                Amberose
NEWDATA                                       0
USERID         954c2bf7a96d309c7bd4fe0dd0c1290f
id                                     26416603
name                                       萨利机长
type                             剧情|灾难|冒险|传记|历史
actors         汤姆·汉克斯|艾伦·艾克哈特|劳拉·琳妮|杰瑞·费拉拉|安娜·冈
region                                       美国
director                              克林特·伊斯特伍德
trait                                  感人|经典|励志
rat                                         8.3
enable                                        1
description                                 NaN
img                                         NaN
userid         954c2bf7a96d309c7bd4fe0dd0c1290f
rmax                                          5
rmin                                    

In [72]:
df.tail(10)

,ID,TIME,MOVIEID,RATING,CREATOR,NEWDATA,USERID,id,name,type,...,enable,description,img,userid,rmax,rmin,ravg,rcount,rsum,rmedian
389679,99662,2016-06-29 18:31:43,1294766.0,2,李琳,0,3631013e410c5be3dc78a5c549a172a0,1294766,泰山,剧情|冒险|爱情,...,1.0,NaN,NaN,3631013e410c5be3dc78a5c549a172a0,2.0,2.0,2.0,1.0,2.0,2.0
389680,99698,2009-09-16 20:53:00,1294766.0,5,Amber哦,0,eec9a7a2f0ee2a154a8c520a90b946c0,1294766,泰山,剧情|冒险|爱情,...,1.0,NaN,NaN,eec9a7a2f0ee2a154a8c520a90b946c0,5.0,5.0,5.0,1.0,5.0,5.0
389681,99748,2011-01-30 19:47:44,1294766.0,5,Cynth!a,0,85c75e080de34c580075e1263dce1482,1294766,泰山,剧情|冒险|爱情,...,1.0,NaN,NaN,85c75e080de34c580075e1263dce1482,5.0,5.0,5.0,1.0,5.0,5.0
389682,99810,2012-05-22 22:34:17,1294766.0,5,何以解,0,ff00b86015b157cf21d9a75363d9e9b3,1294766,泰山,剧情|冒险|爱情,...,1.0,NaN,NaN,ff00b86015b157cf21d9a75363d9e9b3,5.0,5.0,5.0,1.0,5.0,5.0
389683,99815,2009-09-10 23:26:51,1294766.0,5,蒲公英的着落地,0,8d781f83de391e49624dc5eb9fb47b04,1294766,泰山,剧情|冒险|爱情,...,1.0,NaN,NaN,8d781f83de391e49624dc5eb9fb47b04,5.0,5.0,5.0,1.0,5.0,5.0
389684,99819,2008-12-03 17:03:22,1294766.0,5,就爱看电影,0,f2bfecac902e4a165cd975b21ebdc688,1294766,泰山,剧情|冒险|爱情,...,1.0,NaN,NaN,f2bfecac902e4a165cd975b21ebdc688,5.0,5.0,5.0,1.0,5.0,5.0
389685,99849,2011-02-20 16:17:46,1294766.0,5,Tour-Z,0,2cb93b57d3466df772945386e74d0867,1294766,泰山,剧情|冒险|爱情,...,1.0,NaN,NaN,2cb93b57d3466df772945386e74d0867,5.0,5.0,5.0,1.0,5.0,5.0
389686,99861,2008-03-25 23:24:05,1294766.0,5,γυn jiε,0,60d3c3287fb731c2dca571e9255ce5c8,1294766,泰山,剧情|冒险|爱情,...,1.0,NaN,NaN,60d3c3287fb731c2dca571e9255ce5c8,5.0,5.0,5.0,1.0,5.0,5.0
389687,99862,2012-05-23 11:11:56,1294766.0,4,雪2@1,0,c7749ca4436c8903f2e37c9c9f3e7850,1294766,泰山,剧情|冒险|爱情,...,1.0,NaN,NaN,c7749ca4436c8903f2e37c9c9f3e7850,4.0,4.0,4.0,1.0,4.0,4.0
389688,99869,2011-03-15 14:09:27,1294766.0,4,无眉道人,0,194aba5c2c7da0d3a1c1dd2855cd5a99,1294766,泰山,剧情|冒险|爱情,...,1.0,NaN,NaN,194aba5c2c7da0d3a1c1dd2855cd5a99,4.0,4.0,4.0,1.0,4.0,4.0


In [0]:
df = df.reset_index(drop=True)

In [77]:
df.tail(10)

,ID,TIME,MOVIEID,RATING,CREATOR,NEWDATA,USERID,id,name,type,...,enable,description,img,userid,rmax,rmin,ravg,rcount,rsum,rmedian
289218,99662,2016-06-29 18:31:43,1294766.0,2,李琳,0,3631013e410c5be3dc78a5c549a172a0,1294766,泰山,剧情|冒险|爱情,...,1.0,NaN,NaN,3631013e410c5be3dc78a5c549a172a0,2.0,2.0,2.0,1.0,2.0,2.0
289219,99698,2009-09-16 20:53:00,1294766.0,5,Amber哦,0,eec9a7a2f0ee2a154a8c520a90b946c0,1294766,泰山,剧情|冒险|爱情,...,1.0,NaN,NaN,eec9a7a2f0ee2a154a8c520a90b946c0,5.0,5.0,5.0,1.0,5.0,5.0
289220,99748,2011-01-30 19:47:44,1294766.0,5,Cynth!a,0,85c75e080de34c580075e1263dce1482,1294766,泰山,剧情|冒险|爱情,...,1.0,NaN,NaN,85c75e080de34c580075e1263dce1482,5.0,5.0,5.0,1.0,5.0,5.0
289221,99810,2012-05-22 22:34:17,1294766.0,5,何以解,0,ff00b86015b157cf21d9a75363d9e9b3,1294766,泰山,剧情|冒险|爱情,...,1.0,NaN,NaN,ff00b86015b157cf21d9a75363d9e9b3,5.0,5.0,5.0,1.0,5.0,5.0
289222,99815,2009-09-10 23:26:51,1294766.0,5,蒲公英的着落地,0,8d781f83de391e49624dc5eb9fb47b04,1294766,泰山,剧情|冒险|爱情,...,1.0,NaN,NaN,8d781f83de391e49624dc5eb9fb47b04,5.0,5.0,5.0,1.0,5.0,5.0
289223,99819,2008-12-03 17:03:22,1294766.0,5,就爱看电影,0,f2bfecac902e4a165cd975b21ebdc688,1294766,泰山,剧情|冒险|爱情,...,1.0,NaN,NaN,f2bfecac902e4a165cd975b21ebdc688,5.0,5.0,5.0,1.0,5.0,5.0
289224,99849,2011-02-20 16:17:46,1294766.0,5,Tour-Z,0,2cb93b57d3466df772945386e74d0867,1294766,泰山,剧情|冒险|爱情,...,1.0,NaN,NaN,2cb93b57d3466df772945386e74d0867,5.0,5.0,5.0,1.0,5.0,5.0
289225,99861,2008-03-25 23:24:05,1294766.0,5,γυn jiε,0,60d3c3287fb731c2dca571e9255ce5c8,1294766,泰山,剧情|冒险|爱情,...,1.0,NaN,NaN,60d3c3287fb731c2dca571e9255ce5c8,5.0,5.0,5.0,1.0,5.0,5.0
289226,99862,2012-05-23 11:11:56,1294766.0,4,雪2@1,0,c7749ca4436c8903f2e37c9c9f3e7850,1294766,泰山,剧情|冒险|爱情,...,1.0,NaN,NaN,c7749ca4436c8903f2e37c9c9f3e7850,4.0,4.0,4.0,1.0,4.0,4.0
289227,99869,2011-03-15 14:09:27,1294766.0,4,无眉道人,0,194aba5c2c7da0d3a1c1dd2855cd5a99,1294766,泰山,剧情|冒险|爱情,...,1.0,NaN,NaN,194aba5c2c7da0d3a1c1dd2855cd5a99,4.0,4.0,4.0,1.0,4.0,4.0


In [78]:
df.head(1).T

,0
ID,10000
TIME,2015-10-09 20:02:19
MOVIEID,1.05339e+07
RATING,4
CREATOR,妖怪dodo
NEWDATA,0
USERID,cf2349f9c01f9a5cd4050aebd30ab74f
id,10533913
name,头脑特工队
type,剧情|奇幻|冒险|喜剧


In [0]:
df_main = df.drop(['name', 'CREATOR','description','img','ID','NEWDATA','MOVIEID'], axis=1)

In [98]:
df_main.shape

(289228, 18)

In [99]:
df_main.head().T

,0,1,2,3,4
TIME,2015-10-09 20:02:19,2009-01-31 22:36:03,2016-11-25 17:19:18,2016-01-03 09:56:48,2016-03-20 22:54:09
RATING,4,4,2,4,4
USERID,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f
id,10533913,1294829,22939161,25864085,26303622
type,剧情|奇幻|冒险|喜剧,剧情|喜剧|爱情,奇幻|战争|剧情|科幻|冒险|动作,剧情|灾难|冒险|喜剧|科幻,剧情|犯罪|传记|喜剧|历史
actors,艾米·波勒|菲利丝·史密斯|理查德·坎德|比尔·哈德尔|刘易斯·布莱克,罗伯特·德尼罗|本·斯蒂勒|特莉·保罗|布莱思·丹纳|欧文·威尔逊,克里斯·派恩|扎克瑞·昆图|佐伊·索尔达娜|西蒙·佩吉|安东·叶利钦,马特·达蒙|杰西卡·查斯坦|克里斯汀·韦格|杰夫·丹尼尔斯|迈克尔·佩纳,克里斯蒂安·贝尔|史蒂夫·卡瑞尔|瑞恩·高斯林|布拉德·皮特|梅丽莎·里奥
region,美国,美国,美国,英国,美国
director,彼特·道格特|罗纳尔多·德尔·卡门,杰伊·罗奇,林诣彬,雷德利·斯科特,亚当·麦凯
trait,感人|经典|励志,经典|搞笑,经典,经典|励志,经典
rat,8.7,7.1,7.3,8.4,8.3


In [0]:
df_main = df_main.rename(columns = {'id':'movieid'})

In [101]:
df_main.shape

(289228, 18)

In [0]:
df_main = df_main.drop(['enable'], axis=1)

In [104]:
df_main.head().T

,0,1,2,3,4
TIME,2015-10-09 20:02:19,2009-01-31 22:36:03,2016-11-25 17:19:18,2016-01-03 09:56:48,2016-03-20 22:54:09
RATING,4,4,2,4,4
USERID,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f
movieid,10533913,1294829,22939161,25864085,26303622
type,剧情|奇幻|冒险|喜剧,剧情|喜剧|爱情,奇幻|战争|剧情|科幻|冒险|动作,剧情|灾难|冒险|喜剧|科幻,剧情|犯罪|传记|喜剧|历史
actors,艾米·波勒|菲利丝·史密斯|理查德·坎德|比尔·哈德尔|刘易斯·布莱克,罗伯特·德尼罗|本·斯蒂勒|特莉·保罗|布莱思·丹纳|欧文·威尔逊,克里斯·派恩|扎克瑞·昆图|佐伊·索尔达娜|西蒙·佩吉|安东·叶利钦,马特·达蒙|杰西卡·查斯坦|克里斯汀·韦格|杰夫·丹尼尔斯|迈克尔·佩纳,克里斯蒂安·贝尔|史蒂夫·卡瑞尔|瑞恩·高斯林|布拉德·皮特|梅丽莎·里奥
region,美国,美国,美国,英国,美国
director,彼特·道格特|罗纳尔多·德尔·卡门,杰伊·罗奇,林诣彬,雷德利·斯科特,亚当·麦凯
trait,感人|经典|励志,经典|搞笑,经典,经典|励志,经典
rat,8.7,7.1,7.3,8.4,8.3


In [0]:
#pd.to_datetime(df_main['TIME'], format='%d%b%Y:%H:%M:%S.%f')

In [126]:
datetime.datetime.strptime(df['TIME'][0],'%Y-%m-%d %H:%M:%S').year - 2000

15

In [0]:
df_main = df_main.dropna(subset=['USERID'])

In [0]:
index = 0

In [0]:
def process_time(t):
  global index
  index = index + 1
  try:
    return datetime.datetime.strptime(t,'%Y-%m-%d %H:%M:%S').year - 2000
  except:
    print(index)

In [0]:
df_main['TIME_DIS'] = df_main['TIME'].apply(lambda x: process_time(x))

In [162]:
df_main.dropna(subset=['USERID']).shape

(289187, 18)

In [160]:
df_main.head().T

,0,1,2,3,4
TIME,2015-10-09 20:02:19,2009-01-31 22:36:03,2016-11-25 17:19:18,2016-01-03 09:56:48,2016-03-20 22:54:09
RATING,4,4,2,4,4
USERID,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f,cf2349f9c01f9a5cd4050aebd30ab74f
movieid,10533913,1294829,22939161,25864085,26303622
type,剧情|奇幻|冒险|喜剧,剧情|喜剧|爱情,奇幻|战争|剧情|科幻|冒险|动作,剧情|灾难|冒险|喜剧|科幻,剧情|犯罪|传记|喜剧|历史
actors,艾米·波勒|菲利丝·史密斯|理查德·坎德|比尔·哈德尔|刘易斯·布莱克,罗伯特·德尼罗|本·斯蒂勒|特莉·保罗|布莱思·丹纳|欧文·威尔逊,克里斯·派恩|扎克瑞·昆图|佐伊·索尔达娜|西蒙·佩吉|安东·叶利钦,马特·达蒙|杰西卡·查斯坦|克里斯汀·韦格|杰夫·丹尼尔斯|迈克尔·佩纳,克里斯蒂安·贝尔|史蒂夫·卡瑞尔|瑞恩·高斯林|布拉德·皮特|梅丽莎·里奥
region,美国,美国,美国,英国,美国
director,彼特·道格特|罗纳尔多·德尔·卡门,杰伊·罗奇,林诣彬,雷德利·斯科特,亚当·麦凯
trait,感人|经典|励志,经典|搞笑,经典,经典|励志,经典
rat,8.7,7.1,7.3,8.4,8.3


In [161]:
df_main.shape

(289187, 18)

In [0]:
df_main = df_main.drop(['TIME'], axis=1)

In [165]:
df_main.head().T

,RATING,USERID,movieid,type,actors,region,director,trait,rat,userid,rmax,rmin,ravg,rcount,rsum,rmedian,TIME_DIS
0,4,cf2349f9c01f9a5cd4050aebd30ab74f,10533913,剧情|奇幻|冒险|喜剧,艾米·波勒|菲利丝·史密斯|理查德·坎德|比尔·哈德尔|刘易斯·布莱克,美国,彼特·道格特|罗纳尔多·德尔·卡门,感人|经典|励志,8.7,cf2349f9c01f9a5cd4050aebd30ab74f,5.0,2.0,3.85714,7.0,27.0,4.0,15
1,4,cf2349f9c01f9a5cd4050aebd30ab74f,1294829,剧情|喜剧|爱情,罗伯特·德尼罗|本·斯蒂勒|特莉·保罗|布莱思·丹纳|欧文·威尔逊,美国,杰伊·罗奇,经典|搞笑,7.1,cf2349f9c01f9a5cd4050aebd30ab74f,5.0,2.0,3.85714,7.0,27.0,4.0,9
2,2,cf2349f9c01f9a5cd4050aebd30ab74f,22939161,奇幻|战争|剧情|科幻|冒险|动作,克里斯·派恩|扎克瑞·昆图|佐伊·索尔达娜|西蒙·佩吉|安东·叶利钦,美国,林诣彬,经典,7.3,cf2349f9c01f9a5cd4050aebd30ab74f,5.0,2.0,3.85714,7.0,27.0,4.0,16
3,4,cf2349f9c01f9a5cd4050aebd30ab74f,25864085,剧情|灾难|冒险|喜剧|科幻,马特·达蒙|杰西卡·查斯坦|克里斯汀·韦格|杰夫·丹尼尔斯|迈克尔·佩纳,英国,雷德利·斯科特,经典|励志,8.4,cf2349f9c01f9a5cd4050aebd30ab74f,5.0,2.0,3.85714,7.0,27.0,4.0,16
4,4,cf2349f9c01f9a5cd4050aebd30ab74f,26303622,剧情|犯罪|传记|喜剧|历史,克里斯蒂安·贝尔|史蒂夫·卡瑞尔|瑞恩·高斯林|布拉德·皮特|梅丽莎·里奥,美国,亚当·麦凯,经典,8.3,cf2349f9c01f9a5cd4050aebd30ab74f,5.0,2.0,3.85714,7.0,27.0,4.0,16
